In [ ]:
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from tensorflow import keras
from sklearn import model_selection
from sklearn.metrics import classification_report
import imgaug.augmenters as iaa
import imgaug as ia

In [ ]:
#iamge augmenation
ia.seed(1)
def Data_Aug(samples):
    randomaug = iaa.Sometimes(
        0.5,
        iaa.OneOf([
            iaa.Sequential([
                iaa.AllChannelsCLAHE(clip_limit=(1, 10)),
                iaa.Affine(rotate=(-5, 5))
            ]),
            iaa.Sequential([
                iaa.Affine(translate_percent={
                    "x": (-0.2, 0.2),
                    "y": (-0.2, 0.2)
                }),
                iaa.Multiply((0.7, 1.3)),
                iaa.AdditiveLaplaceNoise(scale=0.03 * 255, per_channel=True),
                iaa.Fliplr(0.3),
                iaa.Flipud(0.3)
            ]),
            iaa.Sequential([
                iaa.Affine(rotate=(-10, 10)),
                iaa.Multiply((0.7, 1.3)),
                iaa.AdditiveLaplaceNoise(scale=0.03 * 255, per_channel=True),
                iaa.Fliplr(0.3),
                iaa.Flipud(0.3)
            ]),
            iaa.Sequential([
                iaa.PerspectiveTransform(scale=(0, 0.15)),
                iaa.Multiply((0.7, 1.3)),
                iaa.AdditiveLaplaceNoise(scale=0.03 * 255, per_channel=True),
                iaa.Fliplr(0.3),
                iaa.Flipud(0.3)
            ])
        ]))


    return randomaug.augment_images(samples)

def resize_images(imgs, hight, width):
    seq = iaa.Resize({"height": hight, "width": width})
    aug = seq.augment_images(imgs)
    return aug

In [ ]:

class cifar10data(keras.utils.Sequence):
    '''
    generate data batch of cifar10
    10 classes
    '''
    def __init__(self, data_x, data_y, batch_size, train):
        self.data_x = data_x
        self.data_y = data_y
        self.train = train
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.data_y) / self.batch_size))

    def __getitem__(self, idx):

        batch_x = self.data_x[idx * self.batch_size:(idx + 1) *
                              self.batch_size]
        batch_y = self.data_y[idx * self.batch_size:(idx + 1) *
                              self.batch_size]
        if self.train == True:
            batch_x = Data_Aug(batch_x)

        batch_x = np.clip(batch_x / 255.0, 0, 1).astype("float32")
        batch_y = keras.utils.to_categorical(np.array(batch_y), 10)
        return batch_x, [batch_y, batch_y, batch_y, batch_y, batch_y]

In [ ]:
#callback for save model
class CustomEarlyStoppingAndSave(keras.callbacks.Callback):
    def __init__(self, save_path, patience=0):
        super(CustomEarlyStoppingAndSave, self).__init__()
        self.patience = patience
        self.save_path = save_path
        self.best_weights = None

    def on_train_begin(self, logs=None):
        self.wait = 0
        self.stopped_epoch = 0
        #self.best_val_distance_loss = np.Inf
        self.best_val_acc = 0

    def on_epoch_end(self, epoch, logs=None):
        #val_distance_loss = logs.get('val_MMD_loss')
        val_acc = logs.get('val_model_accuracy') + logs.get(
            'val_model_1_accuracy') + logs.get(
                'val_model_2_accuracy') + logs.get(
                    'val_model_3_accuracy') + logs.get('val_model_4_accuracy')

        if  np.greater(val_acc, self.best_val_acc):
            #self.best_val_distance_loss = val_distance_loss
            self.best_val_acc = val_acc
            self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
            self.stopped_epoch = epoch
        #else:
            #self.wait += 1
            #if self.wait >= self.patience:
                #self.stopped_epoch = epoch
                #self.model.stop_training = True
                #print("Restoring model weights from the end of the best epoch.")
                #self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        self.model.set_weights(self.best_weights)
        if self.stopped_epoch > 0:
            print("Epoch %05d: early stopping and save the model" %
                  (self.stopped_epoch + 1))
        self.model.save(self.save_path)

In [ ]:
#distance loss
def descriptor(X, mask = False, max_norm = False):
  X = tf.reduce_mean(X, axis=-1)
  if mask:
    mean_X = tf.reduce_mean(X, axis = [1,2], keepdims= True)
    X = tf.where(X > mean_X, X , tf.zeros_like(X, dtype=tf.float32))
  if max_norm:
    max_X = tf.reduce_max(X, axis = [1,2], keepdims= True)
    X = tf.math.divide_no_nan(X, max_X)
  return X 
  
def compute_euclidean_sum(X, Y):
  L2_distance = tf.reduce_mean(tf.exp(-(X - Y)**2), axis= [1,2])
  return tf.reduce_mean(L2_distance)

def compute_cosine_sum(X, Y):
  X = tf.reshape(X, [tf.shape(X)[0], -1])
  Y = tf.reshape(Y, [tf.shape(Y)[0], -1])
  loss = -keras.losses.cosine_similarity(X,Y,axis=-1)
  return tf.reduce_mean(loss)

def cosine_euclidean_sum_loss(x, cosine_weight, euclidean_weight, mask, max_norm):
  descriptor0 = descriptor(x[0], mask=mask, max_norm=max_norm)
  descriptor1 = descriptor(x[1], mask=mask, max_norm=max_norm)
  descriptor2 = descriptor(x[2], mask=mask, max_norm=max_norm)
  descriptor3 = descriptor(x[3], mask=mask, max_norm=max_norm)
  descriptor4 = descriptor(x[4], mask=mask, max_norm=max_norm)
  cosine_sum = compute_cosine_sum(descriptor0, descriptor1)+compute_cosine_sum(descriptor0, descriptor2)+compute_cosine_sum(descriptor0, descriptor3)+compute_cosine_sum(descriptor0, descriptor4)+compute_cosine_sum(descriptor1, descriptor2)+compute_cosine_sum(descriptor1, descriptor3)+compute_cosine_sum(descriptor1, descriptor4)+compute_cosine_sum(descriptor2, descriptor3)+compute_cosine_sum(descriptor2, descriptor4)+compute_cosine_sum(descriptor3, descriptor4)
  euclidean_sum = compute_euclidean_sum(descriptor0, descriptor1)+compute_euclidean_sum(descriptor0, descriptor2)+compute_euclidean_sum(descriptor0, descriptor3)+compute_euclidean_sum(descriptor0, descriptor4)+compute_euclidean_sum(descriptor1, descriptor2)+compute_euclidean_sum(descriptor1, descriptor3)+compute_euclidean_sum(descriptor1, descriptor4)+compute_euclidean_sum(descriptor2, descriptor3)+compute_euclidean_sum(descriptor2, descriptor4)+compute_euclidean_sum(descriptor3, descriptor4)
  return cosine_sum, euclidean_sum, cosine_weight*cosine_sum+euclidean_weight*euclidean_sum  

In [ ]:
#create single base Alex architecture
def create_AlexNet(input_shape, num_classes=10, random_seed=None):
  inputs = keras.Input(shape=input_shape)
  x = keras.layers.Conv2D(96, (3,3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(inputs)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x)
  x = keras.layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)
  
  x = keras.layers.Conv2D(256, (3,3), padding='same', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  x = keras.layers.BatchNormalization()(x)
  x = keras.layers.Activation('relu')(x) 
  x = keras.layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)
  
 
  x = keras.layers.Conv2D(384, (3,3), padding='same', activation='relu', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x) 
  x = keras.layers.Conv2D(384, (3,3), padding='same', activation='relu', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x) 
  x = keras.layers.Conv2D(256, (3,3), padding='same', activation='relu', kernel_initializer=keras.initializers.HeNormal(seed=random_seed))(x)
  feature_maps = keras.layers.MaxPooling2D(pool_size=(3,3),strides=(2,2),padding='same')(x)

  x = keras.layers.Flatten()(feature_maps)
  #x = keras.layers.Dense(2048,activation='relu')(x)
  #x = keras.layers.Dropout(0.5)(x)
  x = keras.layers.Dense(128,activation='relu')(x)
  x = keras.layers.Dropout(0.5)(x)
  outputs = keras.layers.Dense(num_classes, activation='softmax')(x)
  model = keras.Model(inputs=inputs, outputs=[outputs,feature_maps])
  return model

In [ ]:
#create five base models for training 
input_size = (32, 32, 3) 
model0 = create_AlexNet(input_shape=input_size, num_classes=10,  random_seed=1)
model1 = create_AlexNet(input_shape=input_size, num_classes=10,  random_seed=2)
model2 = create_AlexNet(input_shape=input_size, num_classes=10,  random_seed=3)
model3 = create_AlexNet(input_shape=input_size, num_classes=10,  random_seed=4)
model4 = create_AlexNet(input_shape=input_size, num_classes=10,  random_seed=5)
allinputs = keras.Input(shape=input_size)
output0, feature_maps0 = model0(allinputs)
output1, feature_maps1 = model1(allinputs)
output2, feature_maps2 = model2(allinputs)
output3, feature_maps3 = model3(allinputs)
output4, feature_maps4 = model4(allinputs)
model_train = keras.Model(
    inputs=allinputs, outputs=[output0, output1, output2, output3, output4])

#implement distance loss
loss1, loss2, loss = cosine_euclidean_sum_loss(x=[feature_maps0, feature_maps1, feature_maps2, feature_maps3, feature_maps4], 
                                      cosine_weight=0.5, euclidean_weight=0.05, mask=True, max_norm=False)

model_train.add_loss(loss)
model_train.add_metric(loss1, name="loss1", aggregation='mean')
model_train.add_metric(loss2, name="loss2", aggregation='mean')


model_train.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                    loss=['categorical_crossentropy','categorical_crossentropy','categorical_crossentropy','categorical_crossentropy','categorical_crossentropy'],
                    metrics=['accuracy'])

#load training, validation and testing samples
(x_origin_train, y_origin_train), (x_test,y_test) = keras.datasets.cifar10.load_data()

y_origin_train = y_origin_train.reshape(-1,)
y_test = y_test.reshape(-1,)

x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x_origin_train,
    y_origin_train,
    test_size=10000,
    random_state=0,
    stratify=y_origin_train)
_, x_val, _, y_val = model_selection.train_test_split(
    x_val,
    y_val,
    test_size=2000,
    random_state=0,
    stratify=y_val)

#randomly select training samples
train_index = []
random.seed(0)
for i in range(10):
    train_index += (random.sample(list(np.where(y_train == i)[0]), 10)) # number of training samples per class
random.shuffle(train_index)
sampled_x_train = x_train[train_index]
sampled_y_train = y_train[train_index]
del train_index

train_sequence = cifar10data(sampled_x_train, sampled_y_train, batch_size=10, train=True)
validation_sequence = cifar10data(x_val, y_val, batch_size=10, train=False)

#train base models and save parameters
history = model_train.fit(
    train_sequence,
    epochs=300,
    validation_data=validation_sequence,
    #validation_steps=200,
    callbacks=CustomEarlyStoppingAndSave(
        save_path='100_samples_seed_fix.h5',
        patience=10))
frame = pd.DataFrame({
    'loss':
    history.history['loss'],
    'loss1':
    history.history['loss1'],
    'loss2':
    history.history['loss2'],
    'model_0_loss':
    history.history['model_loss'],
    'model_1_loss':
    history.history['model_1_loss'],
    'model_2_loss':
    history.history['model_2_loss'],
    'model_3_loss':
    history.history['model_3_loss'],
    'model_4_loss':
    history.history['model_4_loss'],
    'model_0_accuracy':
    history.history['model_accuracy'],
    'model_1_accuracy':
    history.history['model_1_accuracy'],
    'model_2_accuracy':
    history.history['model_2_accuracy'],
    'model_3_accuracy':
    history.history['model_3_accuracy'],
    'model_4_accuracy':
    history.history['model_4_accuracy'],
    'val_loss':
    history.history['val_loss'],
    'val_loss1':
    history.history['val_loss1'],
    'val_loss2':
    history.history['val_loss2'],
    'val_model_0_loss':
    history.history['val_model_loss'],
    'val_model_1_loss':
    history.history['val_model_1_loss'],
    'val_model_2_loss':
    history.history['val_model_2_loss'],
    'val_model_3_loss':
    history.history['val_model_3_loss'],
    'val_model_4_loss':
    history.history['val_model_4_loss'],
    'val_model_0_accuracy':
    history.history['val_model_accuracy'],
    'val_model_1_accuracy':
    history.history['val_model_1_accuracy'],
    'val_model_2_accuracy':
    history.history['val_model_2_accuracy'],
    'val_model_3_accuracy':
    history.history['val_model_3_accuracy'],
    'val_model_4_accuracy':
    history.history['val_model_4_accuracy'],
})
#frame.to_excel('/content/drive/MyDrive/cifar10experiment/Alexnet/ns5models300s100samples/table_100_samples_seed_fix.xlsx')


Epoch 1/300
10/10 [==============================] - 23s 430ms/step - loss: 24.4439 - model_loss: 3.4695 - model_1_loss: 3.8658 - model_2_loss: 3.9168 - model_3_loss: 4.1937 - model_4_loss: 3.9914 - model_accuracy: 0.0900 - model_1_accuracy: 0.0900 - model_2_accuracy: 0.0900 - model_3_accuracy: 0.1100 - model_4_accuracy: 0.1200 - loss1: 9.0835 - loss2: 9.2966 - val_loss: 16.9233 - val_model_loss: 2.4679 - val_model_1_loss: 2.4151 - val_model_2_loss: 2.3260 - val_model_3_loss: 2.3539 - val_model_4_loss: 2.3915 - val_model_accuracy: 0.1030 - val_model_1_accuracy: 0.1120 - val_model_2_accuracy: 0.1070 - val_model_3_accuracy: 0.0750 - val_model_4_accuracy: 0.0985 - val_loss1: 8.9684 - val_loss2: 9.6957
Epoch 2/300
10/10 [==============================] - 3s 294ms/step - loss: 18.1889 - model_loss: 2.6920 - model_1_loss: 2.7894 - model_2_loss: 2.5336 - model_3_loss: 2.5895 - model_4_loss: 2.6871 - model_accuracy: 0.1100 - model_1_accuracy: 0.0800 - model_2_accuracy: 0.1400 - model_3_accurac

In [ ]:
#evualate the five base models
x_test = x_test/255.0
evaluate = model_train.evaluate(x_test,[keras.utils.to_categorical(y_test),keras.utils.to_categorical(y_test),keras.utils.to_categorical(y_test),keras.utils.to_categorical(y_test),keras.utils.to_categorical(y_test)],batch_size=10)
evaluate


1000/1000 [==============================] - 13s 11ms/step - loss: 17.1137 - model_loss: 3.1278 - model_1_loss: 3.1341 - model_2_loss: 3.1164 - model_3_loss: 3.2746 - model_4_loss: 3.4791 - model_accuracy: 0.3108 - model_1_accuracy: 0.3211 - model_2_accuracy: 0.3036 - model_3_accuracy: 0.3233 - model_4_accuracy: 0.3063 - loss1: 1.3909 - loss2: 5.7256


In [ ]:
#hard voting ensemble
class Ensemble:
    def __init__(self, model, testdata ,batch_size):
        self.ensemble_model = model
        self.testdata = testdata
        self.batch_size = batch_size
        self.data_size = testdata[1].shape[0]
    def prediction(self):
        predictions =[]
        for i in range(int(self.data_size/self.batch_size)):
            batch_predictions = np.argmax(self.ensemble_model.predict(self.testdata[0][i*self.batch_size:(i+1)*self.batch_size]),axis=2) 
            batch_predictions = np.stack(batch_predictions, axis=1)
            batch_predictions = [np.argmax(np.bincount(pre)) for pre in batch_predictions]
            predictions = predictions+batch_predictions
        return np.array(predictions)

    def evaluate(self):
        predictions = self.prediction()
        accuracy = np.sum(
            predictions == self.testdata[1]) / self.data_size
        return accuracy
AEnsemble = Ensemble(model = model_train,testdata = (x_test, y_test),batch_size=10)
AEnsemble.evaluate()

0.3318

In [ ]:
#soft voting ensemble
def SoftVotingEnsemble(model, x, y):
  data_size = x.shape[0]
  predictions = tf.argmax(tf.reduce_mean(model.predict(x), axis = 0),axis=1)
  accuracy = np.sum(
     predictions == y) / data_size
  return accuracy
SoftVotingEnsemble(model_train, x_test, y_test)

0.3374

In [ ]:
#callback for ensemble learning
class eCustomEarlyStoppingAndSave(keras.callbacks.Callback):
    def __init__(self, save_path, patience=0):
        super(eCustomEarlyStoppingAndSave, self).__init__()
        self.patience = patience
        self.save_path = save_path
        self.best_weights = None

    def on_train_begin(self, logs=None):
        #self.wait = 0
        self.stopped_epoch = 0
        #self.best_val_distance_loss = np.Inf
        self.best_val_acc = 0

    def on_epoch_end(self, epoch, logs=None):
        #val_distance_loss = logs.get('val_MMD_loss')
        val_acc = logs.get('val_accuracy') 

        if  np.greater(val_acc, self.best_val_acc):
            #self.best_val_distance_loss = val_distance_loss
            self.best_val_acc = val_acc
            #self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
            self.stopped_epoch = epoch
        #else:
            #self.wait += 1
            #if self.wait >= self.patience:
                #self.stopped_epoch = epoch
                #self.model.stop_training = True
                #print("Restoring model weights from the end of the best epoch.")
                #self.model.set_weights(self.best_weights)

    def on_train_end(self, logs=None):
        self.model.set_weights(self.best_weights)
        if self.stopped_epoch >= 0:
            print("Epoch %05d: early stopping and save the model" %
                  (self.stopped_epoch + 1))
        #self.model.save(self.save_path)

In [ ]:
#samples for ensemble learning 
class ecifar10data(keras.utils.Sequence):
    '''
    generate data batch of cifar10
    10 classes
    '''
    def __init__(self, data_x, data_y, batch_size, train):
        self.data_x = data_x
        self.data_y = data_y
        self.train = train
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.data_y) / self.batch_size))

    def __getitem__(self, idx):

        batch_x = self.data_x[idx * self.batch_size:(idx + 1) *
                              self.batch_size]
        batch_y = self.data_y[idx * self.batch_size:(idx + 1) *
                              self.batch_size]
        if self.train == True:
            batch_x = Data_Aug(batch_x)

        batch_x = np.clip(batch_x / 255.0, 0, 1).astype("float32")
        batch_y = keras.utils.to_categorical(np.array(batch_y), 10)
        return batch_x, batch_y

In [ ]:
#feature fusion model
model_path = '100_samples_seed_fix.h5'
model = keras.models.load_model(model_path)
extractor_1 = keras.models.Model(inputs=model.layers[1].inputs,outputs=model.layers[1].layers[12].output)
extractor_2 = keras.models.Model(inputs=model.layers[2].inputs,outputs=model.layers[2].layers[12].output)
extractor_3 = keras.models.Model(inputs=model.layers[3].inputs,outputs=model.layers[3].layers[12].output)
extractor_4 = keras.models.Model(inputs=model.layers[4].inputs,outputs=model.layers[4].layers[12].output)
extractor_5 = keras.models.Model(inputs=model.layers[5].inputs,outputs=model.layers[5].layers[12].output)
extractor_1.trainable = False
extractor_2.trainable = False
extractor_3.trainable = False
extractor_4.trainable = False
extractor_5.trainable = False
inputs = keras.Input(shape = input_size)
features_1 = extractor_1(inputs, training = False)
features_2 = extractor_2(inputs, training = False)
features_3 = extractor_3(inputs, training = False)
features_4 = extractor_4(inputs, training = False)
features_5 = extractor_5(inputs, training = False)
x = keras.layers.Concatenate(axis=-1)([features_1,features_2,features_3,features_4,features_5])
#x = keras.layers.add([features_1,features_2,features_3,features_4,features_5])
#x = keras.layers.Conv2D(1024, (1, 1), activation='relu', padding='same',  kernel_initializer=keras.initializers.HeUniform(seed=None))(x)
#x = keras.layers.BatchNormalization()(x)
#x = keras.layers.Activation('relu')(x)
#x = keras.layers.Dropout(0.4)(x)
#x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128,  name='dense_100')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('relu')(x)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(10, activation='softmax', name='dense_101')(x)
ensemble = keras.Model(inputs = inputs, outputs = outputs)
ensemble.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
train_sequence = ecifar10data(sampled_x_train, sampled_y_train, batch_size=10, train=False)
validation_sequence = ecifar10data(x_val, y_val, batch_size=10, train=False)
history = ensemble.fit(
    train_sequence,
    epochs=50,
    validation_data=validation_sequence,
    validation_steps=200,
    callbacks=eCustomEarlyStoppingAndSave(
        save_path='ensemble.h5'))
evaluate = ensemble.evaluate(x_test,keras.utils.to_categorical(y_test),batch_size=10)

Epoch 1/50
10/10 [==============================] - 3s 192ms/step - loss: 2.1789 - accuracy: 0.2400 - val_loss: 7.9155 - val_accuracy: 0.2755
Epoch 2/50
10/10 [==============================] - 1s 147ms/step - loss: 0.9943 - accuracy: 0.7400 - val_loss: 4.4866 - val_accuracy: 0.3050
Epoch 3/50
10/10 [==============================] - 1s 149ms/step - loss: 0.6014 - accuracy: 0.8800 - val_loss: 3.6204 - val_accuracy: 0.3105
Epoch 4/50
10/10 [==============================] - 1s 150ms/step - loss: 0.5133 - accuracy: 0.8600 - val_loss: 3.1717 - val_accuracy: 0.3190
Epoch 5/50
10/10 [==============================] - 1s 149ms/step - loss: 0.3264 - accuracy: 0.9300 - val_loss: 2.9298 - val_accuracy: 0.3315
Epoch 6/50
10/10 [==============================] - 1s 146ms/step - loss: 0.3107 - accuracy: 0.9600 - val_loss: 2.7547 - val_accuracy: 0.3370
Epoch 7/50
10/10 [==============================] - 1s 150ms/step - loss: 0.2816 - accuracy: 0.9300 - val_loss: 2.6365 - val_accuracy: 0.3390
Epoch 

In [ ]:
#fine tuning
extractor_1.trainable = True
extractor_2.trainable = True
extractor_3.trainable = True
extractor_4.trainable = True
extractor_5.trainable = True
ensemble.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
history = ensemble.fit(
    train_sequence,
    epochs=20,
    validation_data=validation_sequence,
    validation_steps=200,
    callbacks=eCustomEarlyStoppingAndSave(
        save_path='ensemble.h5',
        patience=10))
evaluate = ensemble.evaluate(x_test,keras.utils.to_categorical(y_test),batch_size=10)

Epoch 1/20
10/10 [==============================] - 4s 203ms/step - loss: 0.1692 - accuracy: 0.9700 - val_loss: 2.4238 - val_accuracy: 0.3405
Epoch 2/20
10/10 [==============================] - 1s 150ms/step - loss: 0.1467 - accuracy: 0.9800 - val_loss: 2.3554 - val_accuracy: 0.3390
Epoch 3/20
10/10 [==============================] - 1s 150ms/step - loss: 0.1360 - accuracy: 0.9900 - val_loss: 2.3241 - val_accuracy: 0.3360
Epoch 4/20
10/10 [==============================] - 1s 149ms/step - loss: 0.1283 - accuracy: 0.9900 - val_loss: 2.2933 - val_accuracy: 0.3325
Epoch 5/20
10/10 [==============================] - 1s 151ms/step - loss: 0.1381 - accuracy: 0.9900 - val_loss: 2.2812 - val_accuracy: 0.3370
Epoch 6/20
10/10 [==============================] - 1s 150ms/step - loss: 0.1235 - accuracy: 1.0000 - val_loss: 2.2641 - val_accuracy: 0.3345
Epoch 7/20
10/10 [==============================] - 1s 146ms/step - loss: 0.1333 - accuracy: 0.9800 - val_loss: 2.2256 - val_accuracy: 0.3300
Epoch 

In [ ]:
#f1 score
predict = tf.argmax(ensemble.predict(x_test),axis=-1)
classification_report(y_test, predict,digits=4)

'              precision    recall  f1-score   support\n\n           0     0.3626    0.3630    0.3628      1000\n           1     0.4906    0.4420    0.4650      1000\n           2     0.1980    0.1010    0.1338      1000\n           3     0.2235    0.1810    0.2000      1000\n           4     0.2312    0.0800    0.1189      1000\n           5     0.3153    0.3210    0.3181      1000\n           6     0.2649    0.4630    0.3370      1000\n           7     0.3378    0.4300    0.3784      1000\n           8     0.3781    0.4790    0.4226      1000\n           9     0.3943    0.4440    0.4177      1000\n\n    accuracy                         0.3304     10000\n   macro avg     0.3196    0.3304    0.3154     10000\nweighted avg     0.3196    0.3304    0.3154     10000\n'

In [ ]:
#stacking ensemble
model_path = '100_samples_seed_fix.h5'
model = keras.models.load_model(model_path)
model_1 = keras.models.Model(inputs=model.layers[1].inputs,outputs=model.layers[1].outputs[0])
model_2 = keras.models.Model(inputs=model.layers[2].inputs,outputs=model.layers[2].outputs[0])
model_3 = keras.models.Model(inputs=model.layers[3].inputs,outputs=model.layers[3].outputs[0])
model_4 = keras.models.Model(inputs=model.layers[4].inputs,outputs=model.layers[4].outputs[0])
model_5 = keras.models.Model(inputs=model.layers[5].inputs,outputs=model.layers[5].outputs[0])
model_1.trainable = False
model_2.trainable = False
model_3.trainable = False
model_4.trainable = False
model_5.trainable = False
inputs = keras.Input(shape = input_size)
prediction_1 = model_1(inputs, training = False)
prediction_2 = model_2(inputs, training = False)
prediction_3 = model_3(inputs, training = False)
prediction_4 = model_4(inputs, training = False)
prediction_5 = model_5(inputs, training = False)
x = keras.layers.Concatenate(axis=-1)([prediction_1,prediction_2,prediction_3,prediction_4,prediction_5])
x = keras.layers.Dense(128, activation='relu', name='dense_100')(x)
#x = keras.layers.BatchNormalization()(x)
#x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(10, activation='softmax', name='dense_101')(x)
ensemble = keras.Model(inputs = inputs, outputs = outputs)
ensemble.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
history = ensemble.fit(
    train_sequence,
    epochs=50,
    validation_data=validation_sequence,
    validation_steps=200,
    callbacks=eCustomEarlyStoppingAndSave(
        save_path='stacking.h5',
        patience=10))
evaluate = ensemble.evaluate(x_test,keras.utils.to_categorical(y_test),batch_size=10)

Epoch 1/50
10/10 [==============================] - 3s 200ms/step - loss: 2.1568 - accuracy: 0.3400 - val_loss: 2.2580 - val_accuracy: 0.1960
Epoch 2/50
10/10 [==============================] - 1s 152ms/step - loss: 1.8510 - accuracy: 0.8200 - val_loss: 2.2062 - val_accuracy: 0.3000
Epoch 3/50
10/10 [==============================] - 1s 152ms/step - loss: 1.5712 - accuracy: 1.0000 - val_loss: 2.1608 - val_accuracy: 0.3265
Epoch 4/50
10/10 [==============================] - 1s 151ms/step - loss: 1.3049 - accuracy: 1.0000 - val_loss: 2.1227 - val_accuracy: 0.3340
Epoch 5/50
10/10 [==============================] - 1s 142ms/step - loss: 1.0570 - accuracy: 1.0000 - val_loss: 2.0917 - val_accuracy: 0.3340
Epoch 6/50
10/10 [==============================] - 1s 143ms/step - loss: 0.8316 - accuracy: 1.0000 - val_loss: 2.0692 - val_accuracy: 0.3335
Epoch 7/50
10/10 [==============================] - 1s 152ms/step - loss: 0.6334 - accuracy: 1.0000 - val_loss: 2.0569 - val_accuracy: 0.3355
Epoch 

In [ ]:
#finetuning
model_1.trainable = True
model_2.trainable = True
model_3.trainable = True
model_4.trainable = True
model_5.trainable = True
ensemble.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
history = ensemble.fit(
    train_sequence,
    epochs=20,
    validation_data=validation_sequence,
    validation_steps=200,
    callbacks=eCustomEarlyStoppingAndSave(
        save_path='stacking.h5',
        patience=10))
evaluate = ensemble.evaluate(x_test,keras.utils.to_categorical(y_test),batch_size=10)

Epoch 1/20
10/10 [==============================] - 4s 216ms/step - loss: 0.2910 - accuracy: 1.0000 - val_loss: 2.0883 - val_accuracy: 0.3335
Epoch 2/20
10/10 [==============================] - 1s 159ms/step - loss: 0.2877 - accuracy: 1.0000 - val_loss: 2.0978 - val_accuracy: 0.3230
Epoch 3/20
10/10 [==============================] - 1s 155ms/step - loss: 0.2862 - accuracy: 1.0000 - val_loss: 2.1024 - val_accuracy: 0.3240
Epoch 4/20
10/10 [==============================] - 1s 153ms/step - loss: 0.2848 - accuracy: 1.0000 - val_loss: 2.1058 - val_accuracy: 0.3230
Epoch 5/20
10/10 [==============================] - 1s 156ms/step - loss: 0.2836 - accuracy: 1.0000 - val_loss: 2.1092 - val_accuracy: 0.3235
Epoch 6/20
10/10 [==============================] - 1s 158ms/step - loss: 0.2823 - accuracy: 1.0000 - val_loss: 2.1119 - val_accuracy: 0.3235
Epoch 7/20
10/10 [==============================] - 1s 151ms/step - loss: 0.2811 - accuracy: 1.0000 - val_loss: 2.1144 - val_accuracy: 0.3245
Epoch 

In [ ]:
#f1 score
predict = tf.argmax(ensemble.predict(x_test),axis=-1)
classification_report(y_test, predict,digits=4)

'              precision    recall  f1-score   support\n\n           0     0.3376    0.3390    0.3383      1000\n           1     0.5472    0.3710    0.4422      1000\n           2     0.1895    0.1160    0.1439      1000\n           3     0.2028    0.1600    0.1789      1000\n           4     0.2168    0.1500    0.1773      1000\n           5     0.2773    0.3610    0.3136      1000\n           6     0.2920    0.4310    0.3481      1000\n           7     0.4161    0.3870    0.4010      1000\n           8     0.3755    0.5400    0.4430      1000\n           9     0.4217    0.4550    0.4377      1000\n\n    accuracy                         0.3310     10000\n   macro avg     0.3277    0.3310    0.3224     10000\nweighted avg     0.3277    0.3310    0.3224     10000\n'